In [1]:
!python --version

Python 3.12.12


In [2]:
with open('/content/wizard_of_oz.txt','r',encoding='utf-8') as f:
  text = f.read()
print(text[:200])

﻿  DOROTHY AND THE WIZARD IN OZ

  BY

  L. FRANK BAUM

  AUTHOR OF THE WIZARD OF OZ, THE LAND OF OZ, OZMA OF OZ, ETC.

  ILLUSTRATED BY JOHN R. NEILL

  BOOKS OF WONDER WILLIAM MORROW & CO., INC. NEW


In [3]:
chars = sorted(set(text))
print(chars)
print(len(chars))

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']
81


In [4]:
string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

In [5]:
encode('hello')

[61, 58, 65, 65, 68]

In [6]:
decode([44,55,66])

'Tbm'

In [7]:
import torch

In [8]:
data = torch.tensor(encode(text),dtype=torch.long)
print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


In [9]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[:n]

In [10]:
block_size = 8
batch_size = 4

In [11]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print(f'When input is : {context} --> target is {target}')

When input is : tensor([80]) --> target is 1
When input is : tensor([80,  1]) --> target is 1
When input is : tensor([80,  1,  1]) --> target is 28
When input is : tensor([80,  1,  1, 28]) --> target is 39
When input is : tensor([80,  1,  1, 28, 39]) --> target is 42
When input is : tensor([80,  1,  1, 28, 39, 42]) --> target is 39
When input is : tensor([80,  1,  1, 28, 39, 42, 39]) --> target is 44
When input is : tensor([80,  1,  1, 28, 39, 42, 39, 44]) --> target is 32


The langauge models needs to be scaled to process and handle large number of input data without affecting its processing time and performance. <br>
Here , `batch_size` & `block_size` helps in fast training and good performance.

In [12]:
device = 'cuda' if torch.cuda.is_available() else 'CPU'
print(device)


CPU


In [13]:
input = torch.empty(2,3)
input

tensor([[0., 0., 0.],
        [0., 0., 0.]])

### To compare the difference in processing time between CPU and GPU

In [14]:
'''
import time
import numpy as np

torch_rand1 = torch.rand(100,100,100,100).to(device)
torch_tand2 = torch.rand(100,100,100,100).to(device)
np_rand1 = torch.rand(100,100,100,100)
np_rand2 = torch.rand(100,100,100,100)

start_time = time.time()
rand = (torch_rand1 @ torch_tand2)
end_time = time.time()
elapsed_time = end_time - start_time
print(f'Elapsed GPU Processing time : {elapsed_time}')

start_time = time.time()
rand = np.multiply(np_rand1,np_rand2)
end_time = time.time()
elapsed_time = end_time - start_time
print(f'Elapsed CPU Processing time : {elapsed_time}')
'''

"\nimport time\nimport numpy as np\n\ntorch_rand1 = torch.rand(100,100,100,100).to(device)\ntorch_tand2 = torch.rand(100,100,100,100).to(device)\nnp_rand1 = torch.rand(100,100,100,100)\nnp_rand2 = torch.rand(100,100,100,100)\n\nstart_time = time.time()\nrand = (torch_rand1 @ torch_tand2)\nend_time = time.time()\nelapsed_time = end_time - start_time\nprint(f'Elapsed GPU Processing time : {elapsed_time}')\n\nstart_time = time.time()\nrand = np.multiply(np_rand1,np_rand2)\nend_time = time.time()\nelapsed_time = end_time - start_time\nprint(f'Elapsed CPU Processing time : {elapsed_time}')\n"

In [23]:
### Let's do a recap on torch and its function
prob = torch.tensor([0.1,0.9]) #Prob Tensor

# 10% or 0.1 --> 0 , 90% or 0.9 --> 1. Each prob points to the probability of that index in the tensor
# Draw 5 samples
samples = torch.multinomial(prob,num_samples=10,replacement=True)
print(samples)

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])


In [24]:
# Torch concat
tensor = torch.tensor([1,2,3,4,5])
op = torch.cat((tensor,torch.tensor([6,7,8,9])),dim=0)
print(op)

tensor([1, 2, 3, 4, 5, 6, 7, 8, 9])
